# Business Logic
---
This is going to be an experimentation for natural language processing in the context of finance. Using libraries like FinBERT, and LDA, we shall leverage language models to help us inform people about what's going in the world of a specific company. Practices will be followed by extracting the article for a given stock and reusing
the methodddology for analysis here.
---


# Importing Libraries

In [ ]:
!pip install feedparser

In [ ]:
import yfinance as yf
import pandas as pd
from transformers import BertForSequenceClassification, BertTokenizer
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re
import torch

# Text extraction

In [ ]:
class YahooFinanceFullArticleScraper:
    """
    Extracts full article content from Yahoo Finance RSS feeds
    Uses RSS for article discovery, then fetches full content from article URLs
    """

    def __init__(self):
        self.base_rss_url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s={}&region=US&lang=en-US"
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    def _extract_article_text(self, url):
        """Extract full article text from a Yahoo Finance article URL"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find article content - Yahoo Finance uses various selectors
            article_content = None

            # Try common article content selectors
            selectors = [
                'article',
                '[data-module="ArticleBody"]',
                '.caas-body',
                '.article-body',
                '[class*="article"]',
                '[class*="content"]'
            ]

            for selector in selectors:
                article_content = soup.select_one(selector)
                if article_content:
                    break

            if not article_content:
                # Fallback: find main content area
                article_content = soup.find('main') or soup.find('article')

            if article_content:
                # Remove script and style elements
                for script in article_content(["script", "style", "nav", "footer", "header"]):
                    script.decompose()

                # Extract text and clean it
                text = article_content.get_text(separator=' ', strip=True)
                # Clean up multiple whitespaces
                text = re.sub(r'\s+', ' ', text).strip()
                return text

            return None

        except Exception as e:
            return None

    def get_full_articles_for_ticker(self, ticker, max_articles=10, verbose=False):
        """
        Get full article content for a ticker symbol

        Args:
            ticker (str): Stock ticker symbol
            max_articles (int): Maximum number of articles to fetch
            verbose (bool): Print progress messages

        Returns:
            list: List of articles with full text content
        """
        articles = []

        if verbose:
            print(f"Fetching RSS feed for {ticker}...")

        try:
            # Get RSS feed
            feed_url = self.base_rss_url.format(ticker.upper())
            feed = feedparser.parse(feed_url)

            if not feed.entries:
                if verbose:
                    print(f"  ⚠ No articles found for {ticker}")
                return articles

            if verbose:
                print(f"Found {len(feed.entries)} articles in RSS feed")
                print(f"Fetching full content (filtering for articles >150 words)..\n")

            # Process articles until we have max_articles that meet the word count requirement
            articles_processed = 0
            articles_skipped = 0

            for entry in feed.entries:
                # Stop if we have enough articles
                if len(articles) >= max_articles:
                    break

                articles_processed += 1
                article_url = entry.get('link', '').strip()
                title = entry.get('title', '').strip()

                if not article_url:
                    continue

                if verbose:
                    title_short = title[:60] + "..." if len(title) > 60 else title
                    print(f"[{articles_processed}] Fetching: {title_short}...")

                # Extract full article text
                full_text = self._extract_article_text(article_url)

                # Parse publication date
                published = entry.get('published', '')
                published_datetime = None
                if hasattr(entry, 'published_parsed') and entry.published_parsed:
                    try:
                        published_datetime = datetime(*entry.published_parsed[:6])
                    except:
                        pass

                # Calculate word count
                word_count = len(full_text.split()) if full_text else 0

                # Filter: Only keep articles with more than 150 words
                if word_count <= 150:
                    articles_skipped += 1
                    if verbose:
                        print(f"    ⚠ Skipped: {word_count} words (minimum 150 required)")
                    continue

                article = {
                    'ticker': ticker.upper(),
                    'title': title,
                    'link': article_url,
                    'rss_description': entry.get('summary', '').strip(),
                    'published': published,
                    'published_datetime': published_datetime,
                    'guid': entry.get('guid', ''),
                    'full_text': full_text or '',
                    'word_count': word_count,
                    'has_full_text': full_text is not None and len(full_text) > 0
                }

                articles.append(article)

                if verbose and full_text:
                    print(f"    ✓ Retrieved {word_count} words")
                elif verbose:
                    print(f"    ⚠ Could not extract content")

            # Summary
            if verbose:
                print(f"\n📊 Summary:")
                print(f"   Articles processed: {articles_processed}")
                print(f"   Articles skipped (<150 words): {articles_skipped}")
                print(f"   Articles returned: {len(articles)}")

            return articles

        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
            return articles

# Initialize scraper
scraper = YahooFinanceFullArticleScraper()

# Get full articles for a ticker (e.g., 'AAPL')
# The 'articles' variable will contain a list of dictionaries, each with full article content
articles_for_nlp = scraper.get_full_articles_for_ticker('AAPL', max_articles=5, verbose=False)

In [ ]:
articles_for_nlp[0]['full_text']

In [ ]:
articles_for_nlp[4]['full_text']

## Model load

### FinBERT

In [ ]:
model_name = "ProsusAI/finbert"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

# Sentiment analysis

### AAPL

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[0]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[1]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[2]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[3]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[4]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

### Bitcoin (BTC-USD)

In [ ]:
articles_for_nlp = scraper.get_full_articles_for_ticker('BTC-USD', max_articles=5, verbose=False)

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[0]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[1]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[2]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[3]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

In [ ]:
# Extract article text (summary or description, fallback to title)
article_text = articles_for_nlp[4]['full_text']

# Tokenize the extracted text
inputs = tokenizer(article_text, return_tensors="pt", padding=True, truncation=True)

# Pass tokenized input through the model
outputs = model(**inputs)

# Apply softmax to get probabilities
probabilities = torch.softmax(outputs.logits, dim=1)

# Get the predicted sentiment
predicted_class_id = probabilities.argmax().item()
sentiment = model.config.id2label[predicted_class_id]

print(f"Article Text: {article_text}")
print(f"Sentiment Probabilities: {probabilities}")
print(f"Predicted Sentiment: {sentiment}")

# LDA

## Apple (AAPL)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import string

# Download required NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet')

try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

class StockArticleLDA:
    """
    LDA Topic Extraction for Stock News Articles
    """

    def __init__(self, n_topics=3, max_iter=50, random_state=42):
        """
        Initialize LDA analyzer

        Parameters:
        -----------
        n_topics : int, default=3
            Number of topics to extract (recommended 2-4 for 5 articles)
        max_iter : int, default=50
            Maximum iterations for LDA
        random_state : int, default=42
            Random state for reproducibility
        """
        self.n_topics = n_topics
        self.max_iter = max_iter
        self.random_state = random_state
        self.lemmatizer = WordNetLemmatizer()

        # Enhanced stopwords (general + finance-specific)
        self.stop_words = set(stopwords.words('english'))
        finance_stopwords = {
            'said', 'say', 'says', 'company', 'companies', 'stock', 'stocks',
            'share', 'shares', 'market', 'markets', 'new', 'also', 'would',
            'could', 'may', 'might', 'one', 'two', 'first', 'last', 'year',
            'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'
        }
        self.stop_words.update(finance_stopwords)

        self.vectorizer = None
        self.lda_model = None
        self.feature_names = None

    def preprocess_text(self, text):
        """
        Clean and preprocess text

        Parameters:
        -----------
        text : str
            Raw article text

        Returns:
        --------
        str : Cleaned and lemmatized text
        """
        # Convert to lowercase
        text = text.lower()

        # Remove URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Remove email addresses
        text = re.sub(r'\S+@\S+', '', text)

        # Remove special characters and digits (keeping spaces)
        text = re.sub(r'[^a-zA-Z\s]', '', text)

        # Tokenize
        tokens = word_tokenize(text)

        # Remove stopwords, short words, and lemmatize
        cleaned_tokens = []
        for token in tokens:
            if (len(token) >= 3 and
                token not in self.stop_words and
                token not in string.punctuation):
                lemmatized = self.lemmatizer.lemmatize(token, pos='v')  # Verb lemmatization
                lemmatized = self.lemmatizer.lemmatize(lemmatized, pos='n')  # Noun lemmatization
                cleaned_tokens.append(lemmatized)

        return ' '.join(cleaned_tokens)

    def extract_articles(self, articles_dict, n_articles=5):
        """
        Extract and preprocess articles from dictionary

        Parameters:
        -----------
        articles_dict : dict
            Dictionary containing articles with 'full_text' key
        n_articles : int, default=5
            Number of articles to extract

        Returns:
        --------
        list : List of preprocessed article texts
        """
        preprocessed_articles = []

        for i in range(n_articles):
            try:
                raw_text = articles_dict[i]['full_text']
                cleaned_text = self.preprocess_text(raw_text)
                preprocessed_articles.append(cleaned_text)
                print(f"✓ Article {i} preprocessed ({len(cleaned_text)} characters)")
            except (KeyError, IndexError) as e:
                print(f"⚠ Warning: Could not process article {i}: {e}")
                continue

        return preprocessed_articles

    def fit_lda(self, articles, use_tfidf=True, max_features=1000, min_df=1, max_df=0.95):
        """
        Fit LDA model on preprocessed articles

        Parameters:
        -----------
        articles : list
            List of preprocessed article texts
        use_tfidf : bool, default=True
            Use TF-IDF vectorization (recommended for news articles)
        max_features : int, default=1000
            Maximum number of features for vectorization
        min_df : int/float, default=1
            Minimum document frequency
        max_df : float, default=0.95
            Maximum document frequency (remove very common terms)

        Returns:
        --------
        self : Returns instance for method chaining
        """
        if len(articles) < 2:
            raise ValueError("Need at least 2 articles for LDA. Found: {}".format(len(articles)))

        print(f"\n📊 Vectorizing {len(articles)} articles...")

        # Vectorization with TF-IDF or Count
        if use_tfidf:
            self.vectorizer = TfidfVectorizer(
                max_features=max_features,
                min_df=min_df,
                max_df=max_df,
                ngram_range=(1, 2)  # Include bigrams for better topic coherence
            )
        else:
            self.vectorizer = CountVectorizer(
                max_features=max_features,
                min_df=min_df,
                max_df=max_df,
                ngram_range=(1, 2)
            )

        # Transform articles to document-term matrix
        doc_term_matrix = self.vectorizer.fit_transform(articles)
        self.feature_names = self.vectorizer.get_feature_names_out()

        print(f"✓ Document-term matrix shape: {doc_term_matrix.shape}")
        print(f"✓ Vocabulary size: {len(self.feature_names)}")

        # Fit LDA model
        print(f"\n🔍 Fitting LDA with {self.n_topics} topics...")
        self.lda_model = LatentDirichletAllocation(
            n_components=self.n_topics,
            max_iter=self.max_iter,
            random_state=self.random_state,
            learning_method='batch',
            n_jobs=-1
        )

        self.lda_model.fit(doc_term_matrix)

        print(f"✓ LDA model fitted successfully")
        print(f"✓ Log-likelihood: {self.lda_model.score(doc_term_matrix):.2f}")
        print(f"✓ Perplexity: {self.lda_model.perplexity(doc_term_matrix):.2f}")

        return self

    def get_top_words_per_topic(self, n_words=10):
        """
        Extract top words for each topic

        Parameters:
        -----------
        n_words : int, default=10
            Number of top words to extract per topic

        Returns:
        --------
        dict : Dictionary with topic numbers as keys and lists of top words as values
        """
        if self.lda_model is None:
            raise ValueError("Model not fitted. Call fit_lda() first.")

        topics_dict = {}

        for topic_idx, topic in enumerate(self.lda_model.components_):
            top_indices = topic.argsort()[-n_words:][::-1]
            top_words = [self.feature_names[i] for i in top_indices]
            top_weights = [topic[i] for i in top_indices]

            topics_dict[f"Topic {topic_idx + 1}"] = {
                'words': top_words,
                'weights': top_weights
            }

        return topics_dict

    def get_document_topic_distribution(self, articles):
        """
        Get topic distribution for each article

        Parameters:
        -----------
        articles : list
            List of preprocessed article texts

        Returns:
        --------
        numpy.ndarray : Array of shape (n_articles, n_topics) with topic probabilities
        """
        if self.lda_model is None or self.vectorizer is None:
            raise ValueError("Model not fitted. Call fit_lda() first.")

        doc_term_matrix = self.vectorizer.transform(articles)
        topic_distributions = self.lda_model.transform(doc_term_matrix)

        return topic_distributions

    def display_results(self, articles, n_words=10):
        """
        Display comprehensive LDA results

        Parameters:
        -----------
        articles : list
            List of preprocessed article texts
        n_words : int, default=10
            Number of top words to display per topic
        """
        print("\n" + "="*80)
        print("📰 LDA TOPIC EXTRACTION RESULTS")
        print("="*80)

        # Display topics
        topics = self.get_top_words_per_topic(n_words)

        for topic_name, topic_data in topics.items():
            print(f"\n{topic_name}:")
            print("-" * 40)
            for word, weight in zip(topic_data['words'], topic_data['weights']):
                print(f"  {word:20s} {weight:8.4f}")

        # Display document-topic distribution
        print("\n" + "="*80)
        print("📄 ARTICLE-TOPIC DISTRIBUTIONS")
        print("="*80)

        topic_dist = self.get_document_topic_distribution(articles)

        for i, dist in enumerate(topic_dist):
            print(f"\nArticle {i}:")
            for topic_idx, prob in enumerate(dist):
                print(f"  Topic {topic_idx + 1}: {prob:6.2%}")
            dominant_topic = np.argmax(dist) + 1
            print(f"  → Dominant Topic: Topic {dominant_topic}")

        print("\n" + "="*80)


# =============================================================================
# USAGE EXAMPLE
# =============================================================================

def analyze_stock_articles(articles_for_nlp, n_topics=3):
    """
    Main function to analyze stock articles using LDA

    Parameters:
    -----------
    articles_for_nlp : dict
        Dictionary containing articles with structure articles_for_nlp[i]['full_text']
    n_topics : int, default=3
        Number of topics to extract

    Returns:
    --------
    StockArticleLDA : Fitted LDA analyzer object
    """

    # Initialize analyzer
    analyzer = StockArticleLDA(n_topics=n_topics)

    # Extract and preprocess articles
    print("🔄 Extracting and preprocessing articles...")
    print("-" * 80)
    preprocessed_articles = analyzer.extract_articles(articles_for_nlp, n_articles=5)

    if len(preprocessed_articles) < 2:
        print("❌ Error: Need at least 2 valid articles for LDA analysis")
        return None

    # Fit LDA model
    analyzer.fit_lda(preprocessed_articles, use_tfidf=True)

    # Display results
    analyzer.display_results(preprocessed_articles, n_words=10)

    return analyzer


# =============================================================================
# QUICK START - Uncomment to run
# =============================================================================

# Example usage (uncomment when ready to run):
# analyzer = analyze_stock_articles(articles_for_nlp, n_topics=3)

# To get topic keywords programmatically:
# topics = analyzer.get_top_words_per_topic(n_words=10)

# To experiment with different number of topics:
# for k in range(2, 5):
#     print(f"\n{'='*80}\nTesting with {k} topics\n{'='*80}")
#     analyzer = analyze_stock_articles(articles_for_nlp, n_topics=k)

In [ ]:
analyzer = analyze_stock_articles(articles_for_nlp, n_topics=3)

# To get topic keywords programmatically:
# topics = analyzer.get_top_words_per_topic(n_words=10)

# To experiment with different number of topics:
# for k in range(2, 5):
#     print(f"\n{'='*80}\nTesting with {k} topics\n{'='*80}")
#     analyzer = analyze_stock_articles(articles_for_nlp, n_topics=k)